In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib notebook

In [2]:
td = pd.read_csv('../train_cleaned.csv')

In [3]:
td.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Explanation Why the edits make under my surnam...
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,Dew ! He match this background colour I rm see...
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"Hey man , I rm really not try to edit tar It c..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,W More I ca mt make any real suggestion on imp...
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"You , sir , be my hire Any chance you remember..."


In [4]:
with open('compressed_w2v.pkl', 'rb') as f:
    compressed_vectors = pickle.load(f)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#comments = pd.read_csv('comments_en_cleaned.csv')
#comments_en = comments[comments.lang == 'en']
vect = CountVectorizer(ngram_range = (1,1), analyzer = 'word',
                       stop_words = 'english',
                       #max_features = 500,
                       min_df = 10, max_df = 0.95).fit(td.comment_text)
pw = list(vect.vocabulary_.keys())

In [6]:
with open("pw.txt", "w") as text_file:
    for w in pw:
        print(w, file=text_file)

In [7]:
import enchant
c = enchant.Dict("en_UK")
def check_spelling(text):
    if not c.check(text):
        suggestions = list(set(c.suggest(text)).intersection(set(pw)))
        if len(suggestions)>0:
            res = suggestions[0]
        elif len(c.suggest(text))>0:
            res = c.suggest(text)[0]
        else:
            res = text
    else:
        res = text
    return res

In [8]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize,TreebankWordTokenizer
import re
import nltk
from nltk.corpus import wordnet
from emoji.unicode_codes import UNICODE_EMOJI
import emoji
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def clean_comment(text):
    wnl = WordNetLemmatizer()
    tokenizer = TreebankWordTokenizer()
    #deacc = re.sub(r'\!',' exclamation_point ', text)
    tokens = tokenizer.tokenize(text)
    tags = nltk.pos_tag(tokens)
    processed = [wnl.lemmatize(check_spelling(word),get_wordnet_pos(tag)) if get_wordnet_pos(tag)!='' else wnl.lemmatize(check_spelling(word)) for (word, tag) in tags]
#    for (word, tag) in tags:
#        wn_tag = get_wordnet_pos(tag)
#        if wn_tag!='':
#            processed.append(wnl.lemmatize(check_spelling(word),wn_tag))
#        else:
#            processed.append(wnl.lemmatize(check_spelling(word)))
    res = ' '.join(processed)
    return res

In [10]:
clean_comment('thi si osome')

'ti ski use'

In [11]:
tdc = td.sample(frac = 0.1)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
#from keras.layers.cudnn_recurrent import CuDNNLSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import TimeDistributed, Flatten
from keras.models import load_model
from keras.utils import Sequence
from keras.preprocessing.text import Tokenizer
import keras.backend as K
import gensim
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

/home/lsm/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
def make_padded_sequenses(docs, max_length):
    tokens = [doc.split(' ') for doc in docs]
    vecs = [[compressed_vectors[t] if t in compressed_vectors else np.zeros(30) for t in ts] for ts in tokens]
    seqs = np.array([np.pad(np.vstack(v),mode = 'constant', pad_width = ((0,max_length-len(v)),(0,0))) if len(v)<max_length else np.vstack(v)[:max_length,:] for v in vecs])
    return seqs

In [13]:
seqs = make_padded_sequenses(td.cleaned.tolist(),75)

In [14]:
feats = seqs
labels = td.toxic.as_matrix()
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)

In [15]:
class_weight = compute_class_weight('balanced'
                                               ,[0,1]
                                               ,td.toxic.apply(int).tolist())

In [30]:
model = Sequential()
model.add(Conv1D(filters=10, kernel_size=2, input_shape = seqs[0].shape, padding='same', activation='tanh'))
#model.add(Dropout(0.2))
model.add(Conv1D(filters=20, kernel_size=3, padding='same', activation='tanh'))
model.add(MaxPooling1D(pool_size=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
checkpointer = ModelCheckpoint(filepath='weights.hdf5',
                               verbose=1, save_best_only=True,
                               monitor = 'val_loss')
model.fit(X_train, y_train, epochs=10, batch_size=500,
          validation_data = [X_test,y_test], callbacks=[checkpointer],
         class_weight = class_weight)

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 9s 70us/step - loss: 0.1784 - acc: 0.9343 - val_loss: 0.1782 - val_acc: 0.9341

Epoch 00001: val_loss improved from inf to 0.17822, saving model to weights.hdf5
Epoch 2/10
127656/127656 [==============================] - 9s 67us/step - loss: 0.1759 - acc: 0.9356 - val_loss: 0.1760 - val_acc: 0.9347

Epoch 00002: val_loss improved from 0.17822 to 0.17601, saving model to weights.hdf5
Epoch 3/10
127656/127656 [==============================] - 9s 70us/step - loss: 0.1736 - acc: 0.9366 - val_loss: 0.1787 - val_acc: 0.9344

Epoch 00003: val_loss did not improve
Epoch 4/10
127656/127656 [==============================] - 9s 68us/step - loss: 0.1710 - acc: 0.9373 - val_loss: 0.1748 - val_acc: 0.9357

Epoch 00004: val_loss improved from 0.17601 to 0.17478, saving model to weights.hdf5
Epoch 5/10
127656/127656 [==============================] - 9s 71us/step - loss: 0.1693 - acc: 0.938

In [35]:
model.get_weights()[2].shape

(3, 10, 20)